In [1]:
import datetime

import pandas as pd
import spacy
import re
import string

from spacy.tokens import Token
from tqdm import tqdm

from textblob import TextBlob
from textblob import Word

import html

In [2]:
#### Importing the file ####
Path="src/"
Filename='projects_TranslateV2.0.csv'
Cat_File="category_hier.csv"
df=pd.read_csv(Path+Filename)
Cat_data=pd.read_csv(Path+Cat_File)

In [3]:
#Filtering the empty abstracts.
df=df[df["Description"].str.strip()!="No abstract available"]

In [21]:
df['Translates'].head()

0    The work of the research training group is dir...
1    The subject of the research training group 'Mi...
3    The goal of this program consists in the mathe...
4    The aim of the research training group is to p...
7    Ocean engineering encompasses a wide variety o...
Name: Translates, dtype: object

In [54]:
df["Translates"]=df["Translates"].apply(str).apply(html.unescape)

In [63]:
nlp = spacy.load("en_core_web_sm")
ManualStopWords=['e.g.','i.e.','etc.','et al.','ibid.']

def remove_spell_errors(doc):
    bdoc = TextBlob(str(doc))
    ## Correcting the words
    return str(bdoc.correct())

def stop(doc):
    #return [token for token in doc if not token.text in ManualStopWords and len(token.text)>1 and not token.is_digit and not token.is_stop and ( token.text.isalpha() or not token.text.isalnum())]
    return [token for token in doc if not token.text in ManualStopWords and len(token.text)>1 and not token.is_digit and not token.is_stop ]

def lemmatize(doc):
    return [token.lemma_.lower() if token.lemma_ != "-PRON-" else token.text.lower() for token in doc]

def remove_line_breaks(doc):
    return [token.replace("\n", " ").replace("\r", " ") for token in doc]

nlp.add_pipe(stop)
nlp.add_pipe(lemmatize)
nlp.add_pipe(remove_line_breaks)

In [72]:
print(str(datetime.datetime.now())+" : Started preprocessing")

docs = df['Translates'].apply(str).to_list()
processed_docs = []

with tqdm(total=len(docs)) as bar:
    for doc in nlp.pipe(docs):
            line = " ".join(doc)
            ## Removing the punctuation
            line=line.translate(str.maketrans('','',string.punctuation))
            ## Removing numbers 
            line=" ".join(list(filter(lambda w : not w.isdigit(), line.split())))
            processed_docs.append(line)
            bar.update(1)

df["PreProcessedDescription"] = processed_docs
print(str(datetime.datetime.now())+" : Preprocessing completed")

  0%|          | 0/52240 [00:00<?, ?it/s]

2019-11-23 16:02:12.958992 : Started preprocessing


100%|██████████| 52240/52240 [31:50<00:00, 27.34it/s]   

2019-11-23 16:34:03.719295 : Preprocessing completed


In [73]:
df.to_csv(Path+'projects_Preprocessed.csv', index=False)